In [40]:
import numpy as np

def construct_perfect_multiplicative_consistent_ifpr(ifpr_matrix):
    """
    Constructs a perfect multiplicative consistent IFPR matrix using Algorithm I.

    Args:
    - ifpr_matrix: A numpy array of shape (n, n, 2), where each entry (i, j) contains (μ_ij, v_ij).

    Returns:
    - R_bar: The perfect multiplicative consistent IFPR matrix.
    """
    n = ifpr_matrix.shape[0]
    R_bar = np.zeros_like(ifpr_matrix)
    
    for i in range(n):
        for k in range(n):
            if i == k:
                # Diagonal elements should be (0.5, 0.5)
                R_bar[i, k] = (0.5, 0.5)
            elif k > i + 1:
                # Calculate μ_ik and v_ik using the formulas from Algorithm I
                mu_product = np.prod([ifpr_matrix[i, t][0] * ifpr_matrix[t, k][0] for t in range(i + 1, k)])
                v_product = np.prod([ifpr_matrix[i, t][1] * ifpr_matrix[t, k][1] for t in range(i + 1, k)])
                mu_complement_product = np.prod([(1 - ifpr_matrix[i, t][0]) * (1 - ifpr_matrix[t, k][0]) for t in range(i + 1, k)])
                v_complement_product = np.prod([(1 - ifpr_matrix[i, t][1]) * (1 - ifpr_matrix[t, k][1]) for t in range(i + 1, k)])
                
                # Avoid division by zero
                if mu_complement_product == 0 or v_complement_product == 0:
                    R_bar[i, k] = (0.5, 0.5)  # Default to neutral value
                else:
                    mu_ik = np.sqrt(mu_product) / (np.sqrt(mu_product) + np.sqrt(mu_complement_product))
                    v_ik = np.sqrt(v_product) / (np.sqrt(v_product) + np.sqrt(v_complement_product))
                    R_bar[i, k] = (mu_ik, v_ik)
            elif k == i + 1:
                # Copy the original values for k = i + 1
                R_bar[i, k] = ifpr_matrix[i, k]
            else:
                # For k < i, use the reciprocal values
                R_bar[i, k] = (R_bar[k, i][1], R_bar[k, i][0])
    
    return R_bar

def calculate_distance(R_bar, R_p):
    """
    Calculates the distance between two IFPR matrices.

    Args:
    - R_bar: The perfect multiplicative consistent IFPR matrix.
    - R_p: The current IFPR matrix.

    Returns:
    - distance: The distance between R_bar and R_p.
    """
    n = R_bar.shape[0]
    distance_sum = 0
    
    for i in range(n):
        for k in range(n):
            distance_sum += abs(R_bar[i, k][0] - R_p[i, k][0]) + abs(R_bar[i, k][1] - R_p[i, k][1])
    
    return distance_sum / (2 * (n - 1) * (n - 2))

def fuse_ifpr_matrices(R_p, R_bar, sigma):
    """
    Fuses two IFPR matrices using the given sigma parameter.
    Args:
    - R_p: The current IFPR matrix.
    - R_bar: The perfect multiplicative consistent IFPR matrix.
    - sigma: The controlling parameter for fusion.
    Returns:
    - R_tilde: The fused IFPR matrix.
    """
    n = R_p.shape[0]
    R_tilde = np.zeros_like(R_p)
    
    for i in range(n):
        for k in range(n):
            if i == k:
                # Diagonal elements should be (0.5, 0.5)
                R_tilde[i, k] = (0.5, 0.5)
            else:
                mu_p, v_p = R_p[i, k]
                mu_bar, v_bar = R_bar[i, k]
                
                # Avoid division by zero
                mu_p_safe = max(mu_p, 1e-10)
                v_p_safe = max(v_p, 1e-10)
                mu_bar_safe = max(mu_bar, 1e-10)
                v_bar_safe = max(v_bar, 1e-10)
                
                # Fuse membership (μ) and non-membership (v)
                mu_tilde = ((mu_p_safe ** (1 - sigma)) * (mu_bar_safe ** sigma)) / \
                           (((mu_p_safe ** (1 - sigma)) * (mu_bar_safe ** sigma)) +
                            ((1 - mu_p_safe) ** (1 - sigma)) * ((1 - mu_bar_safe) ** sigma))
                v_tilde = ((v_p_safe ** (1 - sigma)) * (v_bar_safe ** sigma)) / \
                          (((v_p_safe ** (1 - sigma)) * (v_bar_safe ** sigma)) +
                           ((1 - v_p_safe) ** (1 - sigma)) * ((1 - v_bar_safe) ** sigma))
                
                # Ensure μ + v ≤ 1
                if mu_tilde + v_tilde > 1:
                    excess = mu_tilde + v_tilde - 1
                    mu_tilde -= excess / 2
                    v_tilde -= excess / 2
                
                R_tilde[i, k] = (mu_tilde, v_tilde)
    
    return R_tilde

def check_and_repair_ifpr(ifpr_matrix, tau=0.1, sigma=0.5, max_iter=100):
    """
    Checks and repairs the IFPR matrix for consistency.
    Args:
    - ifpr_matrix: The input IFPR matrix.
    - tau: The consistency threshold.
    - sigma: The controlling parameter for fusion.
    - max_iter: The maximum number of iterations.
    Returns:
    - R_p: The repaired IFPR matrix.
    """
    n = ifpr_matrix.shape[0]
    p = 1
    R_p = ifpr_matrix.copy()
    
    while p <= max_iter:
        R_bar = construct_perfect_multiplicative_consistent_ifpr(R_p)
        d_R = calculate_distance(R_bar, R_p)
        
        if d_R < tau:
            print("Matrix is consistent")
            break
        
        R_tilde = fuse_ifpr_matrices(R_p, R_bar, sigma)
        R_p = R_tilde
        p += 1
    
    # Validate the repaired IFPR matrix
    for i in range(n):
        for k in range(n):
            mu, v = R_p[i, k]
            if mu + v > 1:
                raise ValueError(f"Invalid IFPR entry at ({i}, {k}): μ + v = {mu + v} > 1")
    
    print(f"Reached maximum iterations ({max_iter}). Returning last repaired matrix.")
    return R_p


ifpr_matrix = np.array([
[[0.5, 0.3], [0.2, 0.6], [0.1, 0.8]],
[[0.6, 0.2], [0.5, 0.3], [0.3, 0.6]],
[[0.8, 0.1], [0.6, 0.3], [0.5, 0.3]]
])

'''
# balanced preference
ifpr_matrix = np.array([
    [[0.5, 0.3], [0.4, 0.4], [0.4, 0.4]],
    [[0.4, 0.4], [0.5, 0.3], [0.4, 0.4]],
    [[0.4, 0.4], [0.4, 0.4], [0.5, 0.3]]
])
'''

'''
# cost focused
ifpr_matrix  = np.array([
    [[0.5, 0.3], [0.7, 0.2], [0.6, 0.3]],
    [[0.2, 0.7], [0.5, 0.3], [0.3, 0.5]],
    [[0.3, 0.6], [0.5, 0.3], [0.5, 0.3]]
])
'''
'''
# Value-focused
ifpr_matrix = np.array([
    [[0.5, 0.3], [0.3, 0.6], [0.4, 0.5]],
    [[0.6, 0.3], [0.5, 0.3], [0.7, 0.2]],
    [[0.5, 0.4], [0.2, 0.7], [0.5, 0.3]]
])
'''

'''
# importance-focused
ifpr_matrix = np.array([
    [[0.5, 0.3], [0.4, 0.5], [0.2, 0.7]],
    [[0.5, 0.4], [0.5, 0.3], [0.3, 0.6]],
    [[0.7, 0.2], [0.6, 0.3], [0.5, 0.3]]
])
'''

'''
# high uncertainty
ifpr_matrix = np.array([
    [[0.5, 0.3], [0.4, 0.4], [0.3, 0.3]],
    [[0.4, 0.4], [0.5, 0.3], [0.4, 0.3]],
    [[0.3, 0.3], [0.3, 0.4], [0.5, 0.3]]
])
'''



# Run consistency check and repair
repaired_matrix = check_and_repair_ifpr(ifpr_matrix)
print("Repaired IFPR Matrix:")
print(repaired_matrix)

Matrix is consistent
Reached maximum iterations (100). Returning last repaired matrix.
Repaired IFPR Matrix:
[[[0.5        0.5       ]
  [0.2        0.6       ]
  [0.16016373 0.71010205]]

 [[0.6        0.2       ]
  [0.5        0.5       ]
  [0.3        0.6       ]]

 [[0.71010205 0.16016373]
  [0.6        0.3       ]
  [0.5        0.5       ]]]


<h2> Now generate weights from the perfect multiplicative consistent intuitionistic
preference relation R matrix that we generated above </h2> 

In [42]:
import numpy as np

def validate_ifpr_matrix(ifpr_matrix):
    """
    Validates that all entries in the IFPR matrix satisfy μ + v ≤ 1.
    """
    n = ifpr_matrix.shape[0]
    for i in range(n):
        for k in range(n):
            mu, v = ifpr_matrix[i, k]
            if mu + v > 1:
                raise ValueError(f"Invalid IFPR entry at ({i}, {k}): μ + v = {mu + v} > 1")

def transform_ifpr_to_ivpr(ifpr_matrix):
    """
    Transforms an IFPR matrix into an Interval-Valued Preference Relation (IVPR).
    Ensures that μ and v are within valid range [0,1] and maintains consistency.
    """
    n = ifpr_matrix.shape[0]
    ivpr_matrix = np.zeros((n, n, 2))
    
    for i in range(n):
        for k in range(n):
            mu_ik, v_ik = ifpr_matrix[i, k]
            mu_ik = max(0, min(mu_ik, 1))
            v_ik = max(0, min(v_ik, 1))
            
            # Ensure μ_ik + (1 - v_ik) does not exceed 1
            upper_bound = min(1, mu_ik + (1 - v_ik))
            ivpr_matrix[i, k] = [mu_ik, upper_bound - mu_ik]
    
    return ivpr_matrix

def calculate_priority_vector(ivpr_matrix):
    """
    Calculates the priority vector while ensuring μ + v ≤ 1 constraint.
    """
    n = ivpr_matrix.shape[0]
    
    total_mu = np.sum(ivpr_matrix[:, :, 0])
    total_v_complement = np.sum(ivpr_matrix[:, :, 1])
    
    if total_mu == 0 or total_v_complement == 0:
        raise ValueError("Division by zero encountered. Check the IFPR matrix for invalid values.")
    
    priority_vector = []
    for i in range(n):
        mu_i = np.sum(ivpr_matrix[i, :, 0]) / total_mu
        v_i = 1 - (np.sum(ivpr_matrix[i, :, 1]) / total_v_complement)
        
        # Ensure μ + v ≤ 1 (proportional adjustment)
        if mu_i + v_i > 1:
            excess = mu_i + v_i - 1
            mu_i -= excess / 2
            v_i -= excess / 2
        
        priority_vector.append((mu_i, v_i))
    
    return priority_vector

def calculate_crisp_weights(priority_vector):
    """
    Computes crisp weights ensuring μ + (1 - μ - v) / 2 is valid.
    """
    crisp_weights = []
    for mu, v in priority_vector:
        crisp_weight = mu + (1 - mu - v) / 2
        crisp_weights.append(crisp_weight)
    
    return crisp_weights


# Validate the repaired IFPR matrix
validate_ifpr_matrix(repaired_matrix)

# Transform IFPR to IVPR
ivpr_matrix = transform_ifpr_to_ivpr(repaired_matrix)

# Calculate the priority vector
priority_vector = calculate_priority_vector(ivpr_matrix)

# Calculate crisp weights
crisp_weights = calculate_crisp_weights(priority_vector)

# Print results
print("Priority Vector (Weights for Each Criteria):")
for i, weight in enumerate(priority_vector):
    print(f"Criterion {i+1}: μ = {weight[0]:.4f}, v = {weight[1]:.4f}")

print("\nCrisp Weights:")
for i, weight in enumerate(crisp_weights):
    print(f"Criterion {i+1}: {weight:.4f}")

Priority Vector (Weights for Each Criteria):
Criterion 1: μ = 0.2113, v = 0.6766
Criterion 2: μ = 0.3440, v = 0.6467
Criterion 3: μ = 0.3840, v = 0.6160

Crisp Weights:
Criterion 1: 0.2673
Criterion 2: 0.3486
Criterion 3: 0.3840


<h2> U + V VALUE FIXED, NOT SURE IF CORRECT BUT WE ROLL WIT IT</h2>

In [44]:
import numpy as np

def transform_ifpr_to_ivpr(ifpr_matrix):
    """
    Transforms an IFPR matrix into an Interval-Valued Preference Relation (IVPR).
    Ensures that μ and v are within valid range [0,1] and maintains consistency.
    """
    n = ifpr_matrix.shape[0]
    ivpr_matrix = np.zeros((n, n, 2))
    
    for i in range(n):
        for k in range(n):
            mu_ik, v_ik = ifpr_matrix[i, k]
            mu_ik = max(0, min(mu_ik, 1))
            v_ik = max(0, min(v_ik, 1))
            
            # Ensure μ_ik + (1 - v_ik) does not exceed 1
            upper_bound = min(1, mu_ik + (1 - v_ik))
            ivpr_matrix[i, k] = [mu_ik, upper_bound - mu_ik]
    print(ivpr_matrix)
    
    return ivpr_matrix

def calculate_priority_vector(ivpr_matrix):
    """
    Calculates the priority vector while ensuring μ + v ≤ 1 constraint.
    """
    n = ivpr_matrix.shape[0]
    
    total_mu = np.sum(ivpr_matrix[:, :, 0])
    total_v_complement = np.sum(ivpr_matrix[:, :, 1])
    
    if total_mu == 0 or total_v_complement == 0:
        raise ValueError("Division by zero encountered. Check the IFPR matrix for invalid values.")
    
    priority_vector = []
    for i in range(n):
        mu_i = np.sum(ivpr_matrix[i, :, 0]) / total_mu
        v_i = 1 - (np.sum(ivpr_matrix[i, :, 1]) / total_v_complement)
        
        # Ensure μ + v ≤ 1
        if mu_i + v_i > 1:
            v_i = 1 - mu_i
        
        priority_vector.append((mu_i, v_i))
    
    return priority_vector

def calculate_crisp_weights(priority_vector):
    """
    Computes crisp weights ensuring μ + (1 - μ - v) / 2 is valid.
    """
    crisp_weights = []
    for mu, v in priority_vector:
        crisp_weight = mu + (1 - mu - v) / 2
        crisp_weights.append(crisp_weight)
    
    return crisp_weights

# Example Input: Replace 'repaired_matrix' with actual IFPR matrix
# repaired_matrix = np.array([...])

# Transform IFPR to IVPR
ivpr_matrix = transform_ifpr_to_ivpr(repaired_matrix)

# Calculate the priority vector
priority_vector = calculate_priority_vector(ivpr_matrix)

# Calculate crisp weights
crisp_weights = calculate_crisp_weights(priority_vector)

# Print results
print("Priority Vector (Weights for Each Criteria):")
for i, weight in enumerate(priority_vector):
    print(f"Criterion {i+1}: μ = {weight[0]:.4f}, v = {weight[1]:.4f}")

print("\nCrisp Weights:")
for i, weight in enumerate(crisp_weights):
    print(f"Criterion {i+1}: {weight:.4f}")


[[[0.5        0.5       ]
  [0.2        0.4       ]
  [0.16016373 0.28989795]]

 [[0.6        0.4       ]
  [0.5        0.5       ]
  [0.3        0.4       ]]

 [[0.71010205 0.28989795]
  [0.6        0.4       ]
  [0.5        0.5       ]]]
Priority Vector (Weights for Each Criteria):
Criterion 1: μ = 0.2113, v = 0.6766
Criterion 2: μ = 0.3440, v = 0.6467
Criterion 3: μ = 0.4447, v = 0.5553

Crisp Weights:
Criterion 1: 0.2673
Criterion 2: 0.3486
Criterion 3: 0.4447


'''

Your code is well-structured and follows the general steps for repairing an Intuitionistic Fuzzy Preference Relation (IFPR) matrix, transforming it into an Interval-Valued Preference Relation (IVPR), calculating the priority vector, and deriving crisp weights. However, there are a few points to consider regarding the correctness of the implementation and the closeness of the weights:

---

### 1. **Cross-Checking Formulas and Implementation**
Let’s verify the key formulas and steps in your code against the theoretical framework:

#### a. **Perfect Multiplicative Consistent IFPR Construction (Algorithm I)**:
   - The formula for constructing the perfect multiplicative consistent IFPR matrix is implemented correctly:
     $$
     \mu_{ik} = \frac{\sqrt{\prod_{t=i+1}^{k-1} \mu_{it} \cdot \mu_{tk}}}{\sqrt{\prod_{t=i+1}^{k-1} \mu_{it} \cdot \mu_{tk}} + \sqrt{\prod_{t=i+1}^{k-1} (1-\mu_{it}) \cdot (1-\mu_{tk})}}
     $$
     Similarly for \(v_{ik}\). This part seems accurate.

#### b. **Distance Calculation**:
   - The distance calculation between two IFPR matrices is implemented as:
     $$
     d(R, R_p) = \frac{1}{2(n-1)(n-2)} \sum_{i=1}^n \sum_{k=1, k \neq i}^n \left( |\mu_{ik} - \mu'_{ik}| + |v_{ik} - v'_{ik}| \right)
     $$
     This is correct for measuring inconsistency.

#### c. **Fusion of IFPR Matrices**:
   - The fusion step uses the parameter \(\sigma\) to combine the current matrix \(R_p\) and the perfect matrix \(R_{\text{bar}}\):
     $$
     \mu_{\text{tilde}} = \frac{(\mu_p^{1-\sigma} \cdot \mu_{\text{bar}}^\sigma)}{(\mu_p^{1-\sigma} \cdot \mu_{\text{bar}}^\sigma) + ((1-\mu_p)^{1-\sigma} \cdot (1-\mu_{\text{bar}})^\sigma)}
     $$
     Similarly for \(v_{\text{tilde}}\). This is also implemented correctly.

#### d. **Priority Vector Calculation**:
   - The priority vector is calculated using the normalized rank sum method:
     $$
     \mu_i = \frac{\sum_{k=1}^n \mu_{ik}}{\sum_{j=1}^n \sum_{k=1}^n \mu_{jk}}, \quad
     v_i = 1 - \frac{\sum_{k=1}^n (1-v_{ik})}{\sum_{j=1}^n \sum_{k=1}^n (1-v_{jk})}
     $$
     This is implemented correctly.

#### e. **Crisp Weights Calculation**:
   - The crisp weights are derived using:
     $$
     w_i = \mu_i + \frac{(1 - \mu_i - v_i)}{2}
     $$
     This formula is standard for converting intuitionistic fuzzy values (IFVs) into crisp weights. It balances membership (\(\mu\)) and non-membership (\(v\)) while considering hesitation (\(1 - \mu - v\)).

---

### 2. **Why Are the Weights So Close?**
The closeness of the weights can be attributed to several factors:

#### a. **Input Matrix Characteristics**:
   - If the input IFPR matrix is already close to being consistent or if the preferences are similar across criteria, the resulting weights will naturally be close. For example, in your input matrix:
     ```
     [[0.5, 0.3], [0.4, 0.4], [0.3, 0.5]],
     [[0.6, 0.3], [0.5, 0.3], [0.4, 0.4]],
     [[0.7, 0.2], [0.6, 0.3], [0.5, 0.3]]
     ```
     The differences between the rows are small, leading to similar weights.

#### b. **Repair Algorithm Behavior**:
   - The repair algorithm may smooth out inconsistencies, further reducing differences between criteria. This is expected behavior but can result in closer weights.

#### c. **Normalization Effect**:
   - The normalization process used to calculate the priority vector ensures that the weights sum to 1. If the raw scores are close, normalization amplifies this effect.

---

### 3. **Suggestions for Improvement**

#### a. **Test with a More Diverse Input Matrix**:
   - To test whether the weights are calculated correctly, use an IFPR matrix with more pronounced differences between criteria. For example:
     ```python
     ifpr_matrix = np.array([
         [[0.5, 0.3], [0.8, 0.1], [0.2, 0.7]],
         [[0.2, 0.8], [0.5, 0.3], [0.9, 0.1]],
         [[0.7, 0.2], [0.1, 0.9], [0.5, 0.3]]
     ])
     ```
     This matrix has larger variations in preferences, which should result in more distinct weights.

#### b. **Adjust the Repair Parameters**:
   - Experiment with different values of \(\tau\) (consistency threshold) and \(\sigma\) (fusion parameter). A smaller \(\tau\) or higher \(\sigma\) might preserve more of the original preferences, leading to more distinct weights.

#### c. **Verify Against Manual Calculations**:
   - Manually calculate the priority vector and crisp weights for a small matrix (e.g., \(3 \times 3\)) to verify that the results match the code output.

#### d. **Consider Alternative Weighting Methods**:
   - If the weights remain too close for VIKOR ranking, consider alternative methods for deriving weights, such as entropy-based weighting or pairwise comparison methods.

---

### 4. **Impact on VIKOR Ranking**
VIKOR is sensitive to differences in weights because it calculates the compromise solution based on weighted distances. If the weights are too close, the ranking might not reflect significant differences between alternatives. To mitigate this:
   - Use a more diverse input matrix.
   - Adjust the repair parameters to preserve more variation.
   - Consider normalizing the weights differently (e.g., scaling them to emphasize differences).

---


In [64]:
import pandas as pd

# Load the dataset
data = pd.read_csv("excel_file/Requirements.csv")
data = data.drop(columns=['Pre-requisite', 'Test Cases'])
# Map Importance values ('H', 'M', 'L') to numerical values (4, 2.5, 1)
importance_mapping = {'H': 9, 'M': 5, 'L': 3}
data['Importance'] = data['Importance'].map(importance_mapping)

# Extract the decision matrix
decision_matrix = data[['Cost', 'Value', 'Importance']].values

# Print the decision matrix
print("Decision Matrix:")
print(decision_matrix)

Decision Matrix:
[[8 9 9]
 [3 2 9]
 [7 8 9]
 [4 3 5]
 [6 7 9]
 [7 8 9]
 [6 3 3]
 [5 4 3]
 [7 1 3]
 [7 3 3]
 [5 6 5]
 [6 4 5]
 [4 3 9]
 [5 7 5]
 [7 2 3]
 [7 6 5]
 [3 2 3]
 [4 5 5]
 [5 4 5]
 [5 4 5]
 [6 5 5]
 [6 6 5]
 [8 6 5]
 [7 5 5]
 [5 9 9]
 [2 6 5]
 [6 9 9]
 [7 8 9]
 [7 4 5]
 [4 8 5]
 [6 6 5]
 [8 5 3]
 [4 9 9]
 [6 8 5]
 [3 5 3]
 [4 8 9]
 [2 4 5]
 [5 8 9]
 [5 8 9]
 [7 7 5]
 [4 8 5]
 [2 8 5]
 [1 7 9]
 [5 6 5]
 [6 4 5]
 [5 9 5]
 [3 8 9]
 [6 7 9]
 [5 8 9]
 [5 7 9]
 [6 6 5]
 [4 8 9]
 [6 5 5]
 [5 3 5]
 [6 4 3]
 [2 7 5]
 [3 6 9]
 [5 7 9]
 [6 8 9]
 [6 7 5]
 [7 5 3]
 [5 7 9]
 [4 3 9]
 [3 6 9]
 [5 3 9]
 [4 3 9]
 [6 4 9]
 [3 3 9]
 [6 4 3]
 [3 1 3]
 [5 3 5]
 [4 6 9]
 [6 6 5]
 [5 7 5]
 [4 4 5]
 [3 4 5]
 [7 5 5]
 [4 3 9]
 [5 7 9]
 [3 3 3]
 [4 2 5]
 [3 2 3]
 [4 6 9]
 [5 5 9]
 [5 5 9]
 [6 8 5]
 [4 7 9]
 [5 6 9]
 [4 5 5]
 [3 7 9]
 [4 6 5]
 [6 9 9]
 [5 5 5]
 [7 8 9]
 [8 8 5]
 [6 4 9]
 [7 7 3]
 [4 8 5]
 [4 5 9]
 [5 4 9]]


In [48]:
def determine_best_and_worst(decision_matrix, criteria_types):
    """
    Determines the best and worst values for each criterion.
    Args:
    - decision_matrix: A numpy array of shape (m, n), where m is the number of alternatives and n is the number of criteria.
    - criteria_types: A list indicating the type of each criterion ('max' for beneficial, 'min' for non-beneficial).
    Returns:
    - f_star: The best values for each criterion.
    - f_minus: The worst values for each criterion.
    """
    f_star = []
    f_minus = []

    for j, criterion_type in enumerate(criteria_types):
        if criterion_type == 'max':  # Beneficial criterion
            f_star.append(np.max(decision_matrix[:, j]))
            f_minus.append(np.min(decision_matrix[:, j]))
        elif criterion_type == 'min':  # Non-beneficial criterion
            f_star.append(np.min(decision_matrix[:, j]))
            f_minus.append(np.max(decision_matrix[:, j]))

    return np.array(f_star), np.array(f_minus)

# Define criteria types
criteria_types = ['min', 'max', 'max']  # Cost: min, Value: max, Importance: max

# Determine best and worst values
f_star, f_minus = determine_best_and_worst(decision_matrix, criteria_types)
print("f_star, f_minus:" , f_star, f_minus)

f_star, f_minus: [1 9 9] [8 1 3]


In [52]:
def substitute_values(decision_matrix, weights, f_star, f_minus):
    """
    Substitutes each value in the decision matrix with the given formula.
    Args:
    - decision_matrix: The decision matrix.
    - weights: The weights for each criterion.
    - f_star: The best values for each criterion.
    - f_minus: The worst values for each criterion.
    Returns:
    - substituted_matrix: The substituted decision matrix.
    """
    m, n = decision_matrix.shape
    substituted_matrix = np.zeros_like(decision_matrix, dtype=float)

    for i in range(m):
        for j in range(n):
            numerator = f_star[j] - decision_matrix[i, j]
            denominator = f_star[j] - f_minus[j] + 1e-10
            substituted_matrix[i, j] = crisp_weights[j] * (numerator / denominator)

    return substituted_matrix

# Substitute values in the decision matrix
substituted_matrix = substitute_values(decision_matrix, crisp_weights, f_star, f_minus)
print("\nSubstituted Decision Matrix:")
print(substituted_matrix)


Substituted Decision Matrix:
[[ 0.2673442   0.          0.        ]
 [ 0.07638406  0.30504178  0.        ]
 [ 0.22915217  0.0435774   0.        ]
 [ 0.11457609  0.26146438  0.29647565]
 [ 0.19096015  0.08715479  0.        ]
 [ 0.22915217  0.0435774   0.        ]
 [ 0.19096015  0.26146438  0.44471348]
 [ 0.15276812  0.21788698  0.44471348]
 [ 0.22915217  0.34861917  0.44471348]
 [ 0.22915217  0.26146438  0.44471348]
 [ 0.15276812  0.13073219  0.29647565]
 [ 0.19096015  0.21788698  0.29647565]
 [ 0.11457609  0.26146438  0.        ]
 [ 0.15276812  0.08715479  0.29647565]
 [ 0.22915217  0.30504178  0.44471348]
 [ 0.22915217  0.13073219  0.29647565]
 [ 0.07638406  0.30504178  0.44471348]
 [ 0.11457609  0.17430959  0.29647565]
 [ 0.15276812  0.21788698  0.29647565]
 [ 0.15276812  0.21788698  0.29647565]
 [ 0.19096015  0.17430959  0.29647565]
 [ 0.19096015  0.13073219  0.29647565]
 [ 0.2673442   0.13073219  0.29647565]
 [ 0.22915217  0.17430959  0.29647565]
 [ 0.15276812  0.          0.     

In [54]:
def calculate_s(substituted_matrix):
    """
    Calculates the utility measure (S_i) for each alternative.
    Args:
    - substituted_matrix: The substituted decision matrix.
    Returns:
    - S: The utility measure for each alternative.
    """
    S = np.sum(substituted_matrix, axis=1)
    return S

# Calculate S_i
S = calculate_s(substituted_matrix)
print("\nUtility Measure (S):", S)


Utility Measure (S): [0.2673442  0.38142583 0.27272957 0.67251612 0.27811494 0.27272957
 0.897138   0.81536858 1.02248483 0.93533003 0.57997596 0.70532278
 0.37604047 0.53639856 0.97890743 0.65636002 0.82613931 0.58536133
 0.66713075 0.66713075 0.66174538 0.61816799 0.69455205 0.69993741
 0.15276812 0.46539987 0.19096015 0.27272957 0.74351481 0.45462914
 0.61816799 0.88636727 0.11457609 0.53101319 0.69540712 0.15815348
 0.55255466 0.19634551 0.19634551 0.61278262 0.45462914 0.37824508
 0.08715479 0.57997596 0.70532278 0.44924377 0.11996145 0.27811494
 0.19634551 0.23992291 0.61816799 0.15815348 0.66174538 0.71070815
 0.85356061 0.42182247 0.20711625 0.23992291 0.23453754 0.57459059
 0.84817524 0.23992291 0.37604047 0.20711625 0.4142325  0.37604047
 0.40884713 0.33784844 0.85356061 0.86971671 0.71070815 0.24530828
 0.61816799 0.53639856 0.62893872 0.59074669 0.69993741 0.37604047
 0.23992291 0.78256192 0.71609352 0.82613931 0.24530828 0.3270777
 0.3270777  0.53101319 0.20173088 0.28350

In [56]:
def calculate_r(substituted_matrix):
    """
    Calculates the regret measure (R_i) for each alternative.
    Args:
    - substituted_matrix: The substituted decision matrix.
    Returns:
    - R: The regret measure for each alternative.
    """
    R = np.max(substituted_matrix, axis=1)
    return R

# Calculate R_i
R = calculate_r(substituted_matrix)
print("\nRegret Measure (R):", R)
len(R)


Regret Measure (R): [0.2673442  0.30504178 0.22915217 0.29647565 0.19096015 0.22915217
 0.44471348 0.44471348 0.44471348 0.44471348 0.29647565 0.29647565
 0.26146438 0.29647565 0.44471348 0.29647565 0.44471348 0.29647565
 0.29647565 0.29647565 0.29647565 0.29647565 0.29647565 0.29647565
 0.15276812 0.29647565 0.19096015 0.22915217 0.29647565 0.29647565
 0.29647565 0.44471348 0.11457609 0.29647565 0.44471348 0.11457609
 0.29647565 0.15276812 0.15276812 0.29647565 0.29647565 0.29647565
 0.08715479 0.29647565 0.29647565 0.29647565 0.07638406 0.19096015
 0.15276812 0.15276812 0.29647565 0.11457609 0.29647565 0.29647565
 0.44471348 0.29647565 0.13073219 0.15276812 0.19096015 0.29647565
 0.44471348 0.15276812 0.26146438 0.13073219 0.26146438 0.26146438
 0.21788698 0.26146438 0.44471348 0.44471348 0.29647565 0.13073219
 0.29647565 0.29647565 0.29647565 0.29647565 0.29647565 0.26146438
 0.15276812 0.44471348 0.30504178 0.44471348 0.13073219 0.17430959
 0.17430959 0.29647565 0.11457609 0.15276

100

In [58]:
def calculate_q(S, R, S_star, S_minus, R_star, R_minus, v=0.5):
    """
    Calculates the compromise solution (Q_i) for each alternative.
    Args:
    - S: The utility measure for each alternative.
    - R: The regret measure for each alternative.
    - S_star, S_minus, R_star, R_minus: Best and worst values for S_i and R_i.
    - v: Strategy weight (default is 0.5).
    Returns:
    - Q: The compromise solution for each alternative.
    """
    m = len(S)
    Q = np.zeros(m)

    for i in range(m):
        s_term = (S[i] - S_star) / (S_minus - S_star + 1e-10)
        r_term = (R[i] - R_star) / (R_minus - R_star + 1e-10)
        Q[i] = v * s_term + (1 - v) * r_term

    return Q

# Calculate S*, R*, S-, R-
S_star = np.min(S)
S_minus = np.max(S)
R_star = np.min(R)
R_minus = np.max(R)

# Calculate Q_i
Q = calculate_q(S, R, S_star, S_minus, R_star, R_minus)

print("\nCompromise Solution (Q):\n", Q)


Compromise Solution (Q):
 [0.35554865 0.46770706 0.30658257 0.61168704 0.2576165  0.30658257
 0.93299327 0.88928173 1.         0.9534096  0.56221778 0.62922452
 0.40567273 0.53892258 0.9767048  0.60305046 0.89503945 0.56509664
 0.60880818 0.60880818 0.60592932 0.58263412 0.6234668  0.62634566
 0.13876483 0.50096877 0.2110261  0.30658257 0.64964086 0.49521105
 0.58263412 0.92723555 0.06650355 0.53604373 0.82515385 0.08979875
 0.54755916 0.16206002 0.16206002 0.57975526 0.49521105 0.45437837
 0.01462106 0.56221778 0.62922452 0.49233219 0.01753748 0.2576165
 0.16206002 0.18535522 0.58263412 0.08979875 0.60592932 0.63210338
 0.90969807 0.47767357 0.1379044  0.18535522 0.2343213  0.55933892
 0.90681921 0.18535522 0.40567273 0.1379044  0.42608907 0.40567273
 0.36405475 0.38525639 0.90969807 0.91833465 0.63210338 0.15832074
 0.58263412 0.53892258 0.58839184 0.5679755  0.62634566 0.40567273
 0.18535522 0.87174425 0.64661058 0.89503945 0.15832074 0.26118774
 0.26118774 0.53604373 0.11309395 0.

In [60]:
def rank_alternatives(Q, S, R):
    """
    Ranks alternatives based on Q, S, and R.
    Args:
    - Q: The compromise solution for each alternative.
    - S: The utility measure for each alternative.
    - R: The regret measure for each alternative.
    Returns:
    - rankings: A list of tuples containing the alternative index and its Q, S, and R values, sorted by Q.
    """
    rankings = sorted(enumerate(zip(Q, S, R)), key=lambda x: (x[1][0], x[1][1], x[1][2]))
    return rankings

# Rank alternatives
rankings = rank_alternatives(Q, S, R)

# Print rankings
print("\nRanked Alternatives:")
for rank, (index, (q, s, r)) in enumerate(rankings, start=1):
    print(f"Rank {rank}: Alternative {index + 1}, Q = {q:.4f}, S = {s:.4f}, R = {r:.4f}")


Ranked Alternatives:
Rank 1: Alternative 43, Q = 0.0146, S = 0.0872, R = 0.0872
Rank 2: Alternative 47, Q = 0.0175, S = 0.1200, R = 0.0764
Rank 3: Alternative 90, Q = 0.0555, S = 0.1635, R = 0.0872
Rank 4: Alternative 33, Q = 0.0665, S = 0.1146, R = 0.1146
Rank 5: Alternative 36, Q = 0.0898, S = 0.1582, R = 0.1146
Rank 6: Alternative 52, Q = 0.0898, S = 0.1582, R = 0.1146
Rank 7: Alternative 87, Q = 0.1131, S = 0.2017, R = 0.1146
Rank 8: Alternative 57, Q = 0.1379, S = 0.2071, R = 0.1307
Rank 9: Alternative 64, Q = 0.1379, S = 0.2071, R = 0.1307
Rank 10: Alternative 25, Q = 0.1388, S = 0.1528, R = 0.1528
Rank 11: Alternative 72, Q = 0.1583, S = 0.2453, R = 0.1307
Rank 12: Alternative 83, Q = 0.1583, S = 0.2453, R = 0.1307
Rank 13: Alternative 38, Q = 0.1621, S = 0.1963, R = 0.1528
Rank 14: Alternative 39, Q = 0.1621, S = 0.1963, R = 0.1528
Rank 15: Alternative 49, Q = 0.1621, S = 0.1963, R = 0.1528
Rank 16: Alternative 50, Q = 0.1854, S = 0.2399, R = 0.1528
Rank 17: Alternative 58, Q 

 Condition Checking 
Acceptable Advantage (C1) :
The difference between the compromise scores of the top two alternatives (Q(A 
2
 )−Q(A 
1
 )) must be greater than or equal to DQ, where:
DQ= 
j−1
1
​
 
Here, j is the total number of alternatives.
Acceptable Stability (C2) :
The best alternative (A 
1
 ) must also rank first in either the utility measure (S 
i
​
 ) or the regret measure (R 
i
​
 ).
If either condition is violated, the compromise solution is extended to include adnal alternatives.</h2>

In [62]:
def check_vikor_conditions(Q, S, R, rankings):
    """
    Checks the VIKOR conditions for acceptable advantage and stability.
    Args:
    - Q: The compromise solution (Q_i) for each alternative.
    - S: The utility measure (S_i) for each alternative.
    - R: The regret measure (R_i) for each alternative.
    - rankings: A list of tuples containing the alternative index and its Q, S, and R values, sorted by Q.
    Returns:
    - compromise_solution: The final compromise solution (list of alternatives).
    - conditions_satisfied: Boolean indicating whether both conditions are satisfied.
    """
    j = len(Q)  # Total number of alternatives
    DQ = 1 / (j - 1)  # Threshold for acceptable advantage

    # Extract the top-ranked alternative (A1) and the second-ranked alternative (A2)
    A1_index, A1_values = rankings[0]
    A2_index, A2_values = rankings[1]

    # Condition 1: Acceptable Advantage
    Q_A1 = A1_values[0]  # Q value of A1
    Q_A2 = A2_values[0]  # Q value of A2
    acceptable_advantage = (Q_A2 - Q_A1) >= DQ

    # Condition 2: Acceptable Stability
    # Check if A1 ranks first in either S or R
    S_rankings = sorted(enumerate(S), key=lambda x: x[1])  # Sort by S_i
    R_rankings = sorted(enumerate(R), key=lambda x: x[1])  # Sort by R_i
    A1_S_rank = next(i for i, (index, _) in enumerate(S_rankings) if index == A1_index)
    A1_R_rank = next(i for i, (index, _) in enumerate(R_rankings) if index == A1_index)
    acceptable_stability = (A1_S_rank == 0) or (A1_R_rank == 0)

    # Determine the compromise solution
    compromise_solution = []
    if acceptable_advantage and acceptable_stability:
        # Both conditions are satisfied: A1 is the best alternative
        compromise_solution.append(A1_index)
        conditions_satisfied = True
    else:
        # One or both conditions are violated: Extend the compromise solution
        conditions_satisfied = False
        for rank, (index, (q, _, _)) in enumerate(rankings):
            if q - Q_A1 < DQ:
                compromise_solution.append(index)

    return compromise_solution, conditions_satisfied


# Example Usage
# Assume Q, S, R, and rankings are already calculated
compromise_solution, conditions_satisfied = check_vikor_conditions(Q, S, R, rankings)

# Print results
if conditions_satisfied:
    print("Both conditions are satisfied.")
    print(f"Best Alternative: {compromise_solution[0] + 1}")
else:
    print("One or both conditions are violated.")
    print("Compromise Solution (Extended):", [idx + 1 for idx in compromise_solution])

One or both conditions are violated.
Compromise Solution (Extended): [43, 47]


In [76]:
data['Q'] = Q
data['S'] = S
data['R'] = R
data['Ranking'] = 0
for rank, (index, _) in enumerate(rankings, start=1):
    data.at[index, 'Ranking'] = rank

# Reorder columns as desired
desired_columns = ['Req ID', 'Req Name', 'Description', 'Cost', 'Value', 'Importance', 'Q', 'S', 'R', 'Ranking']
data = data[desired_columns]
data = data.sort_values(by='Ranking', ascending=True)

data.head()

,Req ID,Req Name,Description,Cost,Value,Importance,Q,S,R,Ranking
42,143,Proceed for Payment,To proceed with the payment process,1,7,9,0.014621,0.087155,0.087155,1
46,147,Payment Timer,Displays the time left to complete the transac...,3,8,9,0.017537,0.119961,0.076384,2
89,190,E-Card category,Choose the category of the e-gift card,3,7,9,0.055454,0.163539,0.087155,3
32,133,Book Ticket,To book tickets for a movie,4,9,9,0.066504,0.114576,0.114576,4
35,136,Select Show Time,To select the desired show time,4,8,9,0.089799,0.158153,0.114576,5


In [78]:
output_file = "excel_file/Ranked_Requirements.csv"
data.to_csv(output_file, index=False)

print(f"\nRanked requirements saved to '{output_file}'")


Ranked requirements saved to 'excel_file/Ranked_Requirements.csv'
